In [ ]:
import sys
#sys.path.append("/home/markdj/repos/Polesia-Landcover/data_stack_testing/")
sys.path.append("C:/Users/tpfdo/OneDrive/Documents/GitHub/Polesia-Landcover/data_stack_testing/")
import ee
import seaborn as sns
from geemap import geemap
from data_stack_tools_v1 import create_data_stack, map_topography
ee.Initialize()

In [ ]:
# Paths and variable settings
#fp_train_ext = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
fp_train_ext = "D:/tpfdo/Documents/Artio_drive/Projects/Polesia/Project_area.shp"
fp_train_points = "D:/tpfdo/Documents/Artio_drive/Projects/Polesia/Training_data/Training_points_500.shp"
label = 'VALUE'
scale = 1000

aoi = geemap.shp_to_ee(fp_train_ext)
start_date_list = ['2018-01-01', '2018-02-01', 
                   '2018-03-01', '2018-04-01', 
                   '2018-05-01', '2018-06-01', 
                   '2018-07-01', '2018-08-01',
                   '2018-09-01', '2018-10-01',
                   '2018-11-01', '2018-12-01']

s2_params = {
    'CLOUD_FILTER': 60,       # int, max cloud coverage (%) permitted in a scene
    'CLD_PRB_THRESH': 40,     # int, 's2cloudless' 'probability' band value > thresh = cloud
    'NIR_DRK_THRESH': 0.15,   # float, if Band 8 (NIR) < NIR_DRK_THRESH = possible shadow
    'CLD_PRJ_DIST': 1,        # int, max distnce [TODO: km or 100m?] from cloud edge for possible shadow 
    'BUFFER': 100             # int, distance (m) used to buffer cloud edges
}

In [ ]:
# Load the data stack
stack = create_data_stack(aoi, start_date_list, s2_params)

band_names = stack.bandNames()
trainingbands = band_names.getInfo()
print('Training bands are:', trainingbands)

In [ ]:
# Load and sample the training data
print('Loading training data...')
training_points = geemap.shp_to_ee(fp_train_points)

data = stack.select(trainingbands).sampleRegions(collection=training_points,
                                                 properties=label,
                                                 scale=scale)

# Remove null value samples
dataNoNulls = data.filter(ee.Filter.notNull(data.first().propertyNames()))

# Split into train and test
split = 0.7
dataNoNulls = dataNoNulls.randomColumn(seed=0)
train = dataNoNulls.filter(ee.Filter.lt('random',split))
test = dataNoNulls.filter(ee.Filter.gte('random',split))

# Set up the classifier
init_params = {"numberOfTrees":100, # the number of individual decision tree models
              "variablesPerSplit":None,  # the number of features to use per split
              "minLeafPopulation":1, # smallest sample size possible per leaf
              "bagFraction":0.5, # fraction of data to include for each individual tree model
              "maxNodes":None, # max number of leafs/nodes per tree
               "seed":0}  # random seed for "random" choices like sampling.

clf = ee.Classifier.smileRandomForest(**init_params).train(train, label, trainingbands)
print('Training data loaded and classifier ready!')

In [ ]:
# Test performance of random forest classifier


In [ ]:
# Use random forest to classify the rest of the stack
print('Using random forest to classify region...')
classified = stack.select(trainingbands).classify(clf)

Map = geemap.Map(center=(52, 28), zoom=8)
Map.addLayer(classified.randomVisualizer(), {}, 'Classified')
Map

Using random forest to classify region...
